Example getting "In Riksdagen kallad" stored in property xxxx to pandas

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-01-13 17:38:06.909399


In [9]:
# function that returns pandas Dataframe
import sys,json
import pandas as pd 
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [11]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#title: Images people with nameform "I Riksdagen kallad"
SELECT DISTINCT ?nameUsedinSwedishPM ?nr ?item ?itemLabel ?img ?svWikipedia WHERE {
  ?item p:P2561 ?nameSwedishPMp.
  OPTIONAL { ?item wdt:P18 ?img. }
  {
    {
      ?nameSwedishPMp ps:P2561 ?nameUsedinSwedishPM;
        pq:P3831 wd:Q110382440.
      OPTIONAL { ?nameSwedishPMp pq:P1545 ?nr. }
    }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
  OPTIONAL {
    ?svWikipedia schema:about ?item;
      schema:inLanguage "sv";
      schema:isPartOf <https://sv.wikipedia.org/>.
  }
}
ORDER BY (?nameUsedinSwedishPM) (?nr)"""




WDRiksdagentot = get_sparql_dataframe(endpoint_url, query)

In [12]:
WDRiksdagentot

,nameUsedinSwedishPM,nr,item,itemLabel,img,svWikipedia
0,Abrahamsson i Jönköping,1,http://www.wikidata.org/entity/Q5627413,Erik Edström,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Erik_Abrahamsson...
1,Adelsköld i Nolhaga o Stockholm,1,http://www.wikidata.org/entity/Q384847,Claes Adolf Adelsköld,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Claes_Adelsk%C3%...
2,Adlersparre i Arvika,2,http://www.wikidata.org/entity/Q5542896,Rudolf Adlersparre,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Rudolf_Adlersparre
3,Adlersparre i Gustafsvik,1,http://www.wikidata.org/entity/Q5542896,Rudolf Adlersparre,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Rudolf_Adlersparre
4,Adolfsson i Göteborg,2,http://www.wikidata.org/entity/Q5410165,Gunnar Adolfsson,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Gunnar_Adolfsson
...,...,...,...,...,...,...
539,Åkerman i Strömstad,None,http://www.wikidata.org/entity/Q6255333,Assar Åkerman,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Assar_%C3%85kerman
540,Öhman i Stockholm,1,http://www.wikidata.org/entity/Q6256712,Gunnar Öhman,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Gunnar_%C3%96hman
541,Öhvall i Bergnäset,None,http://www.wikidata.org/entity/Q6256938,Bertil Öhvall,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Bertil_%C3%96hvall
542,Östling i Valbo,1,http://www.wikidata.org/entity/Q6258086,Olof Östling,http://commons.wikimedia.org/wiki/Special:File...,https://sv.wikipedia.org/wiki/Olof_%C3%96stling


In [13]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-01-13 18:00:05.100066
Time elapsed (hh:mm:ss.ms) 0:21:58.191782
